#OpenAI API 키 설정

* Google Colab 보안 비밀에 OpenAI API 키를 "OPENAI_API_KEY" 라는 이름의 환경 변수로 설정합니다.

In [ ]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY");            # OpenAI API 키


#LangChain 설치



In [ ]:
!pip install langchain>=0.3.0 langchain-openai langchain-community langchain-text-splitters


In [ ]:
!pip install langchain-openai>=0.2.0

In [ ]:
!pip show langchain

#LCEL(LangChain Expression Language)

* 파이프(|) 기호를 사용하여 여러 컴포넌트를 선언적 방식으로 연결하여,  LLM 기반 애플리케이션의 개발을 간소화합니다.
* LCEL의 가장 기본적인 구현은 Prompt template, Chat model, Output parser를 연결하는 것입니다.
* Prompt template, Chat model, Output parser를 연결하면 invoke 메소드로 한번에 모두 순서대로 실행할 수 있습니다.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
import os


# ChatPromptTemplate 객체 생성
prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content="당신은 전문 요리사입니다. 사용자가 요청한 요리의 레시피를 단계별로 작성해 주세요"),
        HumanMessage(content="요리명 : {dish}"),
        AIMessage(content="좋습니다! {dish} 레시피를 단계별로 알려드리겠습니다.")
    ]
)


# Chat Model 생성
model = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    openai_api_key=os.environ["OPENAI_API_KEY"])


# 출력 파서 생성
output_parser = StrOutputParser()


prompt_value = prompt.invoke({"dish": "감자탕"})

ai_message = model.invoke(prompt_value)

output = output_parser.invoke(ai_message)

print(output)



[SystemMessage(content='당신은 전문 요리사입니다. 사용자가 요청한 요리의 레시피를 단계별로 작성해 주세요', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='요리명 : 감자탕', additional_kwargs={}, response_metadata={}),
 AIMessage(content='좋습니다! 감자탕 레시피를 단계별로 알려드리겠습니다.', additional_kwargs={}, response_metadata={})]
messages=[SystemMessage(content='당신은 전문 요리사입니다. 사용자가 요청한 요리의 레시피를 단계별로 작성해 주세요', additional_kwargs={}, response_metadata={}), HumanMessage(content='요리명 : 감자탕', additional_kwargs={}, response_metadata={}), AIMessage(content='좋습니다! 감자탕 레시피를 단계별로 알려드리겠습니다.', additional_kwargs={}, response_metadata={})]
### 감자탕 레시피

#### 재료
- 돼지등뼈 1kg
- 감자 2~3개
- 대파 2대
- 양파 1개
- 마늘 5~6쪽
- 고춧가루 2~3큰술
- 된장 1큰술
- 소금, 후추 적당량
- 물 2리터
- 청양고추 (선택사항) 1~2개
- 들깨가루 (선택사항) 2큰술

#### 조리 도구
- 큰 냄비
- 체
- 나무주걱
- 칼
- 도마

#### 조리 방법

1. **재료 손질하기**
   - 돼지등뼈는 찬물에 30분 정도 담가 핏물을 제거합니다. 이후 흐르는 물에 깨끗이 씻어줍니다.
   - 감자는 껍질을 벗기고 큼직하게 썰어줍니다.
   - 대파는 어슷하게 썰고, 양파는 반으로 자른 후 얇게 썰어줍니다.
   - 마늘은 다져줍니다.
   - 청양고추는 어슷하게 썰어줍니다 (매운 것을 좋아하시면 추가).

2. **육

##Runnable 실행 방법 - invoke, stream, batch, ainvoke
* invoke : 단일 입력에 대해 동기식으로 체인을 실행
* stream : 생성되는 즉시 응답을 순차적으로 반환
* batch : 여러 입력을 한 번에 일괄 처리
* ainvoke : 비동기 방식으로 체인을 실행


In [ ]:
import pprint


prompt_value = prompt.invoke({"dish": "감자탕"})

pprint.pprint(prompt_value.messages)
print('\n\n')


# 체인 생성
chain = prompt | model | output_parser

output = chain.invoke({"dish": "감자탕"})

print(output)

In [ ]:

chain = prompt | model | output_parser

for chunk in chain.stream({"dish": "카레"}):
  print(chunk, end="", flush=True)



In [ ]:


chain = prompt | model | output_parser

outputs = chain.batch([{"dish": "카레"}, {"dish": "우동"}])
print(outputs)


['### 카레 레시피\n\n#### 재료\n- 양파 1개 (중간 크기)\n- 감자 1개 (중간 크기)\n- 당근 1개 (중간 크기)\n- 고기 (닭고기 또는 소고기) 300g\n- 카레 가루 2-3 큰술\n- 식용유 2 큰술\n- 물 3컵\n- 소금, 후추 (기호에 따라)\n- 선택 재료: 완두콩, 버섯, 피망 등\n\n#### 조리 도구\n- 냄비\n- 도마\n- 칼\n- 숟가락\n\n#### 조리 단계\n\n1. **재료 손질하기**\n   - 양파는 잘게 다지고, 감자와 당근은 깍둑썰기로 준비합니다.\n   - 고기는 한 입 크기로 썰어줍니다.\n\n2. **양파 볶기**\n   - 냄비에 식용유를 두르고 중불로 가열합니다.\n   - 다진 양파를 넣고 투명해질 때까지 볶습니다. (약 5분)\n\n3. **고기 추가하기**\n   - 볶은 양파에 썰어놓은 고기를 넣고, 고기가 익을 때까지 볶습니다. (약 5-7분)\n\n4. **채소 넣기**\n   - 고기가 익으면 감자와 당근을 넣고 함께 볶아줍니다. (약 3-5분)\n\n5. **물 붓기**\n   - 재료가 잘 섞이면 물 3컵을 붓고 끓입니다.\n   - 끓기 시작하면 불을 중약불로 줄이고 뚜껑을 덮고 15-20분간 끓입니다. (채소가 부드러워질 때까지)\n\n6. **카레 가루 추가하기**\n   - 채소가 익으면 카레 가루를 넣고 잘 섞어줍니다.\n   - 필요에 따라 소금과 후추로 간을 맞춥니다.\n\n7. **마무리**\n   - 카레가 걸쭉해질 때까지 약 5-10분 더 끓입니다.\n   - 기호에 따라 완두콩이나 버섯 등을 추가할 수 있습니다.\n\n8. **서빙**\n   - 완성된 카레를 그릇에 담고, 밥과 함께 서빙합니다.\n\n맛있게 드세요!', '### 우동 레시피\n\n#### 재료\n- 우동 면 200g\n- 물 1L\n- 대파 1대 (어슷 썰기)\n- 표고버섯 2개 (슬라이스)\n- 시금치 100g\n- 간장 3큰술\n- 미림 1큰술 (선택 사항)\n- 다시마 10g\n- 

In [ ]:
import asyncio

async def main():
    chain = prompt | model | output_parser

    outputs = await chain.ainvoke({"dish": "카레"})
    print(outputs)


await main()

##LCEL의 파이프(|)로 다양한 Runnable 연결하기
* Runnable를 파이프(|)로 연결할 때는 출력 타입과 입력 타입의 일관성에 주의해야 한다.




In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI


# Zero-shot CoT 프롬프트 생성
cot_prompt = ChatPromptTemplate.from_messages(
  [
     SystemMessage(content="당신은 전문 요리사입니다. 사용자가 요청한 요리의 레시피를 단계별로 작성해 주세요"),
     HumanMessage(content="요리명 : {dish}"),
     AIMessage(content="좋습니다! {dish} 레시피를 단계별로 알려드리겠습니다.")
   ]
)

# 모델 생성
    model="gpt-4o-mini",
    temperature=0,
    openai_api_key=os.environ["OPENAI_API_KEY"])


# 출력 파서 생성
output_parser = StrOutputParser()

# 체인 생성(RunnableSequence)
cot_chain  = cot_prompt | model | output_parser

output = cot_chain.invoke({"question": "10 + 2 * 3"})

print(output)


수학 연산에서 곱셈은 덧셈보다 우선적으로 수행됩니다. 따라서 주어진 식 \(10 + 2 * 3\)을 다음과 같이 계산합니다.

1. 먼저 곱셈을 수행합니다: \(2 * 3 = 6\).
2. 그 다음 덧셈을 수행합니다: \(10 + 6 = 16\).

따라서, \(10 + 2 * 3 = 16\)입니다.


In [ ]:
# 단계적으로 생각하는 답변에서 결론을 추출하는 chain을 생성합니다.

summarize_prompt = ChatPromptTemplate.from_messages (
  [
    ("system", "단계적으로 생각한 답변에서 결론만 추출하세요."),
    ("human", "{text}")
  ]
)

# 체인 생성 (RunnableSequence)
summarize_chain = summarize_prompt | model | output_parser




In [ ]:

cot_summarize_chain = cot_chain | summarize_chain

output = cot_summarize_chain.invoke({"question": "10 + 2 * 3"})

print(output)

결론: \(10 + 2 * 3 = 16\)입니다.


##RunnableLambda
* RunnableLambda을 사용하면 사용자 정의 함수나 람다 표현식을 Runnable로 만들어 LCEL 체인의 일부로 통합할 수 있습니다.
* LLM의 응답에 대해 규칙 기반으로 추가 처리를 하거나 어떤 변환을 적용하고 싶은 경우에 사용할 수 있습니다.


LLM이 생성한 텍스트에 대해 소문자를 대문자로 변환하는 처리를 연결하는 Chain 구현 예제

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

prompt = ChatPromptTemplate.from_messages (
  [
      ('system', 'You are a helpful assitant') ,
      ('human', '{input}')
   ]
)


model = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    openai_api_key=os.environ["OPENAI_API_KEY"])


output_parser = StrOutputParser()


In [ ]:
# 소문자를 대문자로 변환하는 함수 구현

from langchain_core.runnables import RunnableLambda


def to_upper(text : str) -> str:
  return text.upper()

# RunnableLambda을 사용하면 사용자 정의 함수를 Runnable로 만들 수 있습니다.
chain = prompt | model | output_parser | RunnableLambda(to_upper)

ai_message = chain.invoke({"input": "Hello"})

print(ai_message)


HELLO! HOW CAN I ASSIST YOU TODAY?


chain 데코레이터를 사용한 RunnableLambda 구현

In [ ]:
from langchain_core.runnables import chain


@chain
def to_upper(text : str) -> str:
  return text.upper()


chain = prompt | model | output_parser | to_upper

ai_message = chain.invoke({"input": "Hello"})

print(ai_message)





HELLO! HOW CAN I ASSIST YOU TODAY?


RunnableLambda 자동 변환


In [ ]:

def to_upper(text : str) -> str:
  return text.upper()

chain = prompt | model | output_parser | to_upper

ai_message = chain.invoke({"input": "안녕하세요"})

print(ai_message)



안녕하세요! 어떻게 도와드릴까요?


##RunnableParallel

- 여러 Runnable을 병렬로 실행하여 그 결과를 하나의 딕셔너리로 묶어 반환하는 구조
- 처리량을 높이고 응답 시간을 단축할 수 있습니다.

[예시] RunnableParallel을 사용하여 사용자가 입력한 주제에 대해 LLM에게 낙관적인 의견과 비관적인 의견을 병렬로 실행하여 생성

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

model = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    openai_api_key=os.environ["OPENAI_API_KEY"])


output_parser = StrOutputParser()


낙관적인 의견을 생성하는 Chain 구현


In [ ]:
optimistic_prompt = ChatPromptTemplate.from_messages(
    {
      ("system", "당신은 낙관주의자입니다. 사용자의 입력에 대해 낙관적인 의견을 제공하세요.") ,
      ("human", "{topic}")
    }
)

# RunnableSequence
optimistic_chain = optimistic_prompt | model | output_parser


비관적인 의견을 생성하는 Chain 구현

In [ ]:
pessimistic_prompt = ChatPromptTemplate.from_messages(
    {
      ("system", "당신은 비관주의자입니다. 사용자의 입력에 대해 비관적인 의견을 제공하세요.") ,
      ("human", "{topic}")
    }
)

# RunnableSequence
pessimistic_chain = pessimistic_prompt | model | output_parser


"RunnableParallel"을 사용하여 낙관적인 의견을 생성하는 optimistic_chain 체인과 비관적인 의견을 생성하는 pessimistic_prompt 체인을 병렬로 실행할 수 있습니다.

In [ ]:
import pprint   # pretty-print

from langchain_core.runnables import RunnableParallel

parallel_chain = RunnableParallel(
    {
      "optimistic_opinion" : optimistic_chain,
      "pessimistic_opinion": pessimistic_chain
    });

output = parallel_chain.invoke({"topic": "생성형 AI"})

pprint.pprint(output)   # 결과를 dict 형태로 반환한다.


{'optimistic_opinion': '생성형 AI는 정말 흥미로운 기술입니다! 이 기술은 창의성과 혁신을 촉진하는 데 큰 역할을 하고 '
                       '있으며, 다양한 분야에서 새로운 가능성을 열어주고 있습니다. 예를 들어, 예술, 음악, 글쓰기 '
                       '등에서 사람들의 상상력을 자극하고, 새로운 아이디어를 탐구하는 데 도움을 줄 수 있습니다. 앞으로 '
                       '생성형 AI가 더욱 발전하면서 우리의 삶을 더욱 풍요롭게 만들어줄 것이라고 믿습니다! 정말 '
                       '기대되지 않나요?',
 'pessimistic_opinion': '생성형 AI는 분명 흥미로운 기술이지만, 그 이면에는 많은 우려가 있습니다. 이 기술이 발전함에 '
                        '따라 인간의 창의성과 노동이 점점 더 대체될 위험이 커지고 있습니다. 결국, 우리는 기계가 '
                        '만들어낸 콘텐츠에 의존하게 될 것이고, 이는 인간의 독창성과 개성을 더욱 퇴색시킬 것입니다. '
                        '또한, 생성형 AI가 잘못된 정보나 편향된 데이터를 생성할 가능성도 높아, 사회적 혼란을 초래할 '
                        '수 있습니다. 결국, 기술이 발전할수록 인간의 삶은 더욱 복잡해지고 불확실해질 것입니다.'}


RunnableParallel의 출력을 Runnable의 입력으로 연결하기

낙관적인 의견과 비판적인 의견을 요약하는 프롬프트를 구현한다.

In [ ]:
synthesize_prompt = ChatPromptTemplate.from_messages(
    [
        ('system', '당신은 객관적인 AI입니다. 두 가지 의견을 종합하세요.') ,
        ('human', '낙관적인 의견 : {optimistic_opinion}\n비판적인 의견 : {pessimistic_opinion}')
    ]
)


In [ ]:

# RunnableSequence object
synthesize_chain = (
    RunnableParallel(
         {
            "optimistic_opinion" : optimistic_chain,
            "pessimistic_opinion": pessimistic_chain
         }
    )
    | synthesize_prompt
    | model
    | output_parser
)

output = synthesize_chain.invoke({"topic": "생성현 AI"})

print(output)



두 가지 의견을 종합해보면, 세상은 긍정적인 가능성과 함께 복잡한 도전 과제를 동시에 안고 있다는 점을 알 수 있습니다. 낙관적인 시각에서는 어려움 속에서도 성장과 배움의 기회를 찾고, 긍정적인 마음가짐이 좋은 결과를 가져올 것이라고 강조합니다. 반면, 비판적인 시각에서는 사회의 갈등과 고립감, 그리고 기대와 실망의 간극을 지적하며, 긍정적인 변화가 드물고 노력의 결과가 헛수고로 돌아갈 수 있음을 경고합니다.

결국, 세상은 긍정적인 가능성과 부정적인 현실이 공존하는 복합적인 환경입니다. 개인의 노력과 긍정적인 태도가 중요하지만, 동시에 사회적 맥락과 현실적인 도전도 인식하고 대응하는 것이 필요합니다. 이러한 균형 잡힌 시각이 더 나은 미래를 위한 길잡이가 될 수 있습니다.


In [ ]:

# RunnableSequence object
synthesize_chain = (
    #RunnableParallel 자동 변환
    {
        "optimistic_opinion" : optimistic_chain,
        "pessimistic_opinion": pessimistic_chain
    }
    | synthesize_prompt
    | model
    | output_parser
)

output = synthesize_chain.invoke({"topic": "생성형 AI"})

print(output)


생성형 AI에 대한 의견은 긍정적인 측면과 비판적인 측면 모두를 포함하고 있습니다. 낙관적인 시각에서는 생성형 AI가 창의성과 혁신을 촉진하며, 다양한 분야에서 사람들의 삶을 풍요롭게 만드는 데 기여할 수 있다고 강조합니다. 이 기술은 반복적인 작업을 자동화하여 사람들이 더 창의적인 작업에 집중할 수 있는 기회를 제공하고, 새로운 아이디어를 제시함으로써 상상력을 자극하는 역할을 합니다.

반면, 비판적인 시각에서는 생성형 AI의 발전이 인간의 창의성과 독창성을 기계에 의존하게 만들 위험이 있으며, 생성된 콘텐츠의 품질과 신뢰성에 대한 우려가 존재한다고 지적합니다. 잘못된 정보나 편향된 데이터로 인해 사회적 혼란이 초래될 수 있으며, 상업적 이익을 위해 남용될 경우 개인의 프라이버시와 권리가 침해될 가능성도 큽니다.

결론적으로, 생성형 AI는 긍정적인 변화와 혁신을 가져올 수 있는 잠재력을 지니고 있지만, 그 이면에는 여러 가지 우려와 위험이 존재합니다. 따라서 이 기술의 발전과 활용에 있어 신중한 접근이 필요하며, 긍정적인 효과를 극대화하고 부정적인 영향을 최소화하기 위한 노력이 중요합니다.


##itemgetter- RunnableLamba와의 조합



In [ ]:
from operator import itemgetter

# "topic"이라는 키의 값을 반환하는 함수를 만들어 topic_getter라는 변수에 저장합니다.
topic_getter = itemgetter("topic")

topic = topic_getter({"topic": "생성형 AI"})

print(topic)



생성형 AI


In [ ]:
from operator import itemgetter


synthesize_prompt = ChatPromptTemplate.from_messages(
    [
        ('system', '당신은 객관적인 AI입니다. {topic}에 대한 두 가지 의견을 종합하세요.') ,
        ('human', '낙관적인 의견 : {optimistic_opinion}\n비판적인 의견 : {pessimistic_opinion}')
    ]
)


# RunnableSequence object
synthesize_chain = (

    # {
    #     "optimistic_opinion": "...긍정 의견...",
    #     "pessimistic_opinion": "...부정 의견...",
    #     "topic": "생성형 AI"
    # }

    #RunnableParallel 자동 변환
    {
        "optimistic_opinion" : optimistic_chain,
        "pessimistic_opinion": pessimistic_chain,
        "topic": itemgetter("topic")  # RullabledLambda('topic')로 자동 변환됨. 입력 딕셔너리에서 'topic' 키의 값을 가져옵니다.
    }
    | synthesize_prompt
    | model
    | output_parser
)

output = synthesize_chain.invoke({"topic": "생성형 AI"})

print(output)



생성형 AI에 대한 의견은 두 가지 상반된 시각을 보여줍니다. 

낙관적인 시각에서는 생성형 AI가 창의성과 혁신을 촉진하며, 예술, 음악, 글쓰기 등 다양한 분야에서 새로운 아이디어를 제공하고 반복적인 작업을 자동화함으로써 사람들의 삶을 풍요롭게 만들 수 있다고 강조합니다. 이 기술은 교육, 의료, 비즈니스 등 여러 분야에서 문제 해결에 기여할 잠재력을 가지고 있으며, 앞으로의 발전이 더욱 효율적이고 창의적인 세상을 만들어 줄 것이라는 기대를 나타냅니다.

반면, 비판적인 시각에서는 생성형 AI의 발전이 인간의 창의성과 독창성을 퇴색시킬 위험이 있으며, AI가 생성한 콘텐츠의 범람이 진정한 예술의 가치를 저하시킬 수 있다고 우려합니다. 또한, AI의 오용 가능성, 즉 잘못된 정보나 편향된 콘텐츠의 확산이 사회적 혼란을 초래할 수 있다는 점을 지적하며, 이러한 기술이 오히려 문제를 야기할 가능성이 높다고 경고합니다.

결국, 생성형 AI는 긍정적인 변화와 함께 여러 가지 우려를 동반하고 있으며, 이 기술의 발전과 활용에 있어 신중한 접근이 필요하다는 점이 공통적으로 나타납니다.


##RunnablePassthrough
* 입력을 그대로 출력하기
* RunnableParallel을 사용할 때 요소 중 일부에서 입력값을 그대로 출력하고 싶은 경우 사용할 수 있습니다.


RunnablePassthrough을 사용하는 RAG의 Chain  구현 예제
* 간단한 웹 검색으로 구현
* Tavily는 AI 에이전트와 LLM(대규모 언어 모델)을 위해 설계된 특화된 검색엔진




TAVILY_API_KEY 환경 변수로 설정

In [ ]:
import os

from google.colab import userdata

os.environ["TAVILY_API_KEY"] = userdata.get("TAVILY_API_KEY");

# print(os.environ["TAVILY_API_KEY"])

tavily-python 패키지 설치

In [ ]:
!pip install tavily-python

In [ ]:
!pip show tavily-python


In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 줄바꿈과 큰따옴표 사용 ;  문맥의 경계를 표시하는 용도
prompt = PromptTemplate.from_template('''
  다음 문맥만 고려하여 질문에 답하세요.
  문맥: """{context}""",
  질문: {question}
''')



model = ChatOpenAI(
    model="gpt-4o-mini",  # 모델
    temperature=0,        # 0 ~ 2 사이에서 선택, 0.8과 같은 높은 값은 무작위하게 출력, 0
    openai_api_key=os.environ["OPENAI_API_KEY"])


Tavily를  LangChain의 Retriever로 사욯하기 위해 TavilySearchAPIRetriever 준비합니다.

In [ ]:
from langchain_community.retrievers import TavilySearchAPIRetriever

# Retriver : Tavily가 웹에서 실시간으로 검색해서 질문(Query)과 가장 관련성이 높은 상위 3개의 문서를 가져옵니다.
retriever = TavilySearchAPIRetriever(k= 3)

In [ ]:
from langchain_core.runnables import RunnablePassthrough

chain = (
    {
        "context": retriever,
        "question": RunnablePassthrough()
    }   # RunnableParallel로 자동 변환됨
    | prompt
    | model
    | StrOutputParser()
)

output = chain.invoke("서울의 현재 날씨는?")

print(output)


서울의 현재 날씨는 부분적으로 흐림(Partly cloudy)이며, 실시간 온도는 5°C, 습도는 75%, 풍속은 5.4 km/h입니다.


assign-RunnableParallel의 출력에 값 추가하기
* retriever의 검색 결과도 Chain 전체의 출력에 포함하고 싶은 경우 RunnablePassthrough의 assign  메소드를 사용합니다.


In [ ]:
import pprint

chain = {
            "question": RunnablePassthrough(),
            "context": retriever,
        } | RunnablePassthrough.assign(
              answer = prompt | model | StrOutputParser()  # 최종 요약된 답변
)

output = chain.invoke("서울의 현재 날씨는?")

pprint.pprint(output)




{'answer': '서울의 현재 날씨는 부분적으로 흐림(Partly cloudy)이며, 실시간 온도는 5°C, 습도는 75%, 풍속은 '
           '5.4 km/h입니다.',
 'context': [Document(metadata={'title': 'Seoul 날씨 상태: 기온 | 30일 예보 - AQI.in', 'source': 'https://www.aqi.in/weather/ko/south-korea/seoul/seoul', 'score': 0.9998535, 'images': []}, page_content='의 현재 일기 예보는 Partly cloudy, 실시간 온도(5°C), 습도 75%, 풍속 5.4 ... Seoul의 월 평균 날씨는 맑은 날 17일, 구름 낀 날 6일, 비 오는 날 6일, 눈'),
             Document(metadata={'title': '서울특별시, 서울시, 대한민국 현재 날씨 - AccuWeather', 'source': 'https://www.accuweather.com/ko/kr/seoul/226081/current-weather/226081', 'score': 0.9994429, 'images': []}, page_content='# 서울특별시, 서울시 현재 위치 사용 48° 서울특별시, 서울시 날씨 오늘 WinterCast 지역 {stormName} 추적기 시간별 일별 레이더 MinuteCast® 월 대기질 건강 및 활동 ### 허리케인 ### 악천후 기상 ### 레이더 및 지도 ### 동영상 오늘   시간별   일별   레이더   MinuteCast®   월   대기질   건강 및 활동 # 현재 기상 RealFeel® 43° RealFeel Shade™ 43° 43° 43° 최대 자외선 지수 동남동 9mi/h 9mi/h 35° F 10mi RealFeel® 65° RealFeel Shade™ 64° 최대 자외선 지수3.0 (보통) 바람남남서 6mi/h 강수 확률25% 강수0.00in Re

assign은 Runnable 인스턴스 메소드로 제공됩니다.

In [ ]:
chain = RunnableParallel({
  "question": RunnablePassthrough(),
  "context": retriever,
}).assign(answer = prompt | model | StrOutputParser())

output = chain.invoke("서울의 현재 날씨는?")

pprint.pprint(output)

{'answer': '서울특별시의 현재 날씨는 83°F(약 28°C)이며, 바람은 서북서 방향으로 시속 4마일(약 6.4km/h)로 불고 '
           '있습니다. 강수 확률은 4%이고, 현재 체감 온도는 88°F(약 31°C)입니다.',
 'context': [Document(metadata={'title': '서울특별시, 서울시, 대한민국 현재 날씨', 'source': 'https://www.accuweather.com/ko/kr/seoul/226081/current-weather/226081', 'score': 0.8568246, 'images': []}, page_content='서울특별시, 서울시, 대한민국 현재 날씨 | AccuWeather 서울특별시, 서울시 ========== 83°F 현재 위치 사용 서울특별시 서울시 83° 서울특별시, 서울시 날씨 오늘WinterCast지역 {stormName} 추적기시간별일별레이더MinuteCast®월대기질건강 및 활동 전 세계 ### 허리케인### 악천후 기상### 레이더 및 지도### 동영상 오늘시간별일별레이더MinuteCast®월대기질건강 및 활동 월요일, 8월 11일 현재 기상 PM 7:48 RealFeel® 88°  88° 서북서 4mi/h 8mi/h 71° F 10mi RealFeel® 81°  바람 남 4mi/h 돌풍 10mi/h 강수 확률 4% 강수 0.00in Image 113 시간 46 분 일몰 PM 7:30 Image 211 시간 58 분 일출 PM 8:40 일몰 AM 8:38 84° 73° 87° 71° 94° 82° ### 시간별### 일별### 월 전 세계 ### 허리케인### 악천후 기상### 레이더 및 지도### 동영상 전 세계아시아대한민국서울시서울특별시 *   동작구, 서울시 *   마포구, 서울시 *   서대문구, 서울시 Image 3Image 4Image 5'),
             Document(metadata={'title': '경기 > 서울 - 오늘의 날

#Chat history
* 챗봇과 같은 대화형 AI 애플리케이션은 대화 이력을 관리해야 합니다.
* LangChain은 Chat History 와 Memory 기능을 통해 이전 대화의 맥락을 유지하고 유기적인 상호작용을 가능하게 합니다.



##Chat history
* LangChain에서 Chat History는 대화 메시지를 물리적인 공간에 저장하고 관리 하는 핵심적인 역할을 수행합니다..
* 사용자와 AI 간의 모든 상호작용을  있는 그대로 저장 관리합니다..


[예시] SQLChatMessageHistory를 사용하여 SQLite를 사용하여 영구적으로 대화 이력 관리
* SQLite는 임베디드 RDBMS입니다.
* 서버를 띄우지 않고 애플리케이션 내부에서 바로 데이터베이스 파일을 다루는 방식입니다.
* 저장 방식에 따라 디스크 기반 RDBMS와 인메모리 기반 RDBMS 가 있습니다.

In [ ]:
# Python 코드로 테이블 생성 (단, 한 번만 실행)

import sqlite3

DB_FILE = "chat_history.db"

# "chat_history.db" 라는 데이터베이스 파일에 연결
conn = sqlite3.connect(DB_FILE)

# SQL 쿼리를 실행하기 위한 커서 객체 생성
cursor = conn.cursor()

cursor.execute("""
  CREATE TABLE IF NOT EXISTS message_store (
      id INTEGER PRIMARY KEY AUTOINCREMENT,
      session_id TEXT NOT NULL,
      message_type TEXT,
      message TEXT NOT NULL,
      additional_kwargs TEXT,  -- LangChain에서 필요할 수 있음 (역할, 모델 파라미터 ...)
      created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
  )
""")

# 테이블 생성 등 변경 사항을 데이터베이스에 저장
conn.commit()

# 연결 종료
conn.close()


In [ ]:
import sqlite3

DB_FILE = "chat_history.db"
conn = sqlite3.connect(DB_FILE)
cursor = conn.cursor()

# PRAGMA 실행
cursor.execute("PRAGMA table_info(message_store);")

# 결과 가져오기
columns = cursor.fetchall()
for col in columns:
    print(col)

conn.close()

(0, 'id', 'INTEGER', 0, None, 1)
(1, 'session_id', 'TEXT', 1, None, 0)
(2, 'message_type', 'TEXT', 0, None, 0)
(3, 'message', 'TEXT', 1, None, 0)
(4, 'additional_kwargs', 'TEXT', 0, None, 0)
(5, 'created_at', 'TIMESTAMP', 0, 'CURRENT_TIMESTAMP', 0)


In [ ]:
from langchain_community.chat_message_histories import SQLChatMessageHistory


# DB 연결 정보 설정
chat_message_histories = SQLChatMessageHistory(
    connection_string=f"sqlite:///{DB_FILE}",
    session_id="session_1234",
    table_name="message_store"
)

# 샘플 데이터(메시지)를 순차적으로 기록한다.

# HumanMessage 추가
human_message = "AI가 무엇인지 기본 개념부터 설명해줘."
chat_message_histories.add_user_message(human_message)

# AIMessage 추가
ai_response_text = "AI는 인간의 학습·추론·판단 능력을 컴퓨터가 수행하도록 만드는 기술을 말합니다."
chat_message_histories.add_ai_message(ai_response_text)

# HumanMessage 추가
human_message = "AI랑 머신러닝은 같은 거야?"
chat_message_histories.add_user_message(human_message)

# AIMessage 추가
ai_response_text = "같지는 않아요. AI가 더 큰 개념이고, 머신러닝은 데이터로 패턴을 학습하는 AI의 한 분야입니다."
chat_message_histories.add_ai_message(ai_response_text)

# HumanMessage 추가
human_message = "AI는 어떤 산업에서 주로 쓰이고 있어?"
chat_message_histories.add_user_message(human_message)

# AIMessage 추가
ai_response_text = "의료, 금융, 제조, 교육, 자율주행, 음성 비서, 추천 시스템 등 거의 모든 산업에서 활용됩니다."
chat_message_histories.add_ai_message(ai_response_text)

# HumanMessage 추가
human_message = "AI가 사람보다 잘하는 분야가 있어?"
chat_message_histories.add_user_message(human_message)

# AIMessage 추가
ai_response_text = "대규모 데이터 분석, 패턴 탐지, 특정 반복 작업 등은 인간보다 훨씬 빠르고 정확하게 처리할 수 있습니다."
chat_message_histories.add_ai_message(ai_response_text)

# HumanMessage 추가
human_message = "반대로 AI가 아직 부족한 부분은 뭐야?."
chat_message_histories.add_user_message(human_message)

# AIMessage 추가
ai_response_text = "상식 기반 판단, 감정 이해, 맥락이 복잡한 의사결정 등은 여전히 어려운 편입니다."
chat_message_histories.add_ai_message(ai_response_text)

# HumanMessage 추가
human_message = "AI가 가지는 위험이나 문제점도 알려줘."
chat_message_histories.add_user_message(human_message)

# AIMessage 추가
ai_response_text = "데이터 편향, 개인정보 침해, 오작동, 자동화로 인한 일자리 감소 등이 대표적인 위험 요소입니다."
chat_message_histories.add_ai_message(ai_response_text)

# HumanMessage 추가
human_message = "AI가 편향된 판단을 하는 이유는 뭔데?"
chat_message_histories.add_user_message(human_message)

# AIMessage 추가
ai_response_text = "AI가 학습하는 데이터 자체에 편견이나 불균형이 있을 경우 그 패턴을 그대로 따라 하기 때문입니다."
chat_message_histories.add_ai_message(ai_response_text)

# HumanMessage 추가
human_message = "AI의 개인정보 문제는 어떻게 발생하는데?"
chat_message_histories.add_user_message(human_message)

# AIMessage 추가
ai_response_text = "AI는 인간의 학습·추론·판단 능력을 컴퓨터가 수행하도록 만드는 기술을 말합니다."
chat_message_histories.add_ai_message(ai_response_text)



print("대화목록 :", chat_message_histories.get_messages())


대화목록 : [HumanMessage(content='AI가 무엇인지 기본 개념부터 설명해줘.', additional_kwargs={}, response_metadata={}), AIMessage(content='AI는 인간의 학습·추론·판단 능력을 컴퓨터가 수행하도록 만드는 기술을 말합니다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='AI랑 머신러닝은 같은 거야?', additional_kwargs={}, response_metadata={}), AIMessage(content='같지는 않아요. AI가 더 큰 개념이고, 머신러닝은 데이터로 패턴을 학습하는 AI의 한 분야입니다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='AI는 어떤 산업에서 주로 쓰이고 있어?', additional_kwargs={}, response_metadata={}), AIMessage(content='의료, 금융, 제조, 교육, 자율주행, 음성 비서, 추천 시스템 등 거의 모든 산업에서 활용됩니다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='AI가 사람보다 잘하는 분야가 있어?', additional_kwargs={}, response_metadata={}), AIMessage(content='대규모 데이터 분석, 패턴 탐지, 특정 반복 작업 등은 인간보다 훨씬 빠르고 정확하게 처리할 수 있습니다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='반대로 AI가 아직 부족한 부분은 뭐야?.', additional_kwargs={}, response_metadata={}), AIMessage(content='상식 기반 판단, 감정 이해, 맥락이 복잡한 의사결정 등은 여전히 어

In [ ]:
import sqlite3


# DB 연결
conn = sqlite3.connect(DB_FILE)

cursor = conn.cursor()

# 테이블의 모든 메시지 정보 조회
cursor.execute("SELECT message FROM message_store")

rows = cursor.fetchall()

# 데이터만 출력
if rows:
    for row in rows:
        print(row)
else:
    print("테이블에 데이터가 없습니다.")

# 연결 종료
conn.close()



In [ ]:
# LangChain 구성 요소
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import AIMessage, HumanMessage


# 모델 설정
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.3,
    openai_api_key=os.environ["OPENAI_API_KEY"])


# 프롬프트 템플릿 생성
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 친절하고 도움이 되는 AI 비서입니다. 이전 대화 기록을 참고하여 사용자 질문에 한국어로 답변해 주세요.") ,
        MessagesPlaceholder(variable_name="chat_history") ,     # 이전 대화 기록이 여기에 들어갑니다.
        ("human", "{input}") ,
    ]
)


# 체인 구성
chain = (
    RunnablePassthrough()   # 입력 값을 그대로 통과
    | prompt
    | llm
    | StrOutputParser()
)


# DB에서 "session_1234" 세션의 Chat History를 조회한다.
messages = chat_message_histories.get_messages();

# 체인 호출
ai_response = chain.invoke({
    "chat_history": messages,
    "input": "AI에 대해 설명해줘."
})


print("\n--- AI 응답 ---")
print(ai_response)


--- AI 응답 ---
AI(인공지능)는 컴퓨터 시스템이 인간의 지능적 행동을 모방하도록 설계된 기술입니다. 이는 학습(데이터로부터 패턴을 인식하고 개선하는 과정), 추론(주어진 정보를 바탕으로 결론을 도출하는 과정), 문제 해결(주어진 문제를 해결하기 위한 방법을 찾는 과정) 등을 포함합니다.

AI는 크게 두 가지로 나눌 수 있습니다:

1. **약한 AI**: 특정 작업을 수행하도록 설계된 AI로, 예를 들어 음성 인식 시스템이나 추천 알고리즘 등이 이에 해당합니다.
2. **강한 AI**: 인간과 유사한 수준의 지능을 갖춘 AI로, 현재는 이론적인 개념에 가까우며 실제로 구현된 사례는 없습니다.

AI는 다양한 산업에서 활용되고 있으며, 데이터 분석, 의료 진단, 자율주행차, 고객 서비스, 금융 거래 등 여러 분야에서 혁신을 가져오고 있습니다. 하지만 데이터 편향, 개인정보 보호 문제, 윤리적 이슈 등 여러 도전 과제도 존재합니다.


##Memory

SQLChatMessageHistory로 DB에서 전체 대화 목록을 불러오고, ConversationSummaryMemory로 요약본을 관리하여, 프롬프트에는 원본 전체 대신 요약(summary)과 최근 N개의 대화 메시지 일부만 전달합니다.

In [ ]:
# Python 코드로 테이블 생성 (단, 한 번만 실행)

import sqlite3

DB_FILE = "chat_history.db"

conn = sqlite3.connect(DB_FILE)

cursor = conn.cursor()

cursor.execute("""
  CREATE TABLE IF NOT EXISTS message_store (
      id INTEGER PRIMARY KEY AUTOINCREMENT,
      session_id TEXT NOT NULL,
      message_type TEXT,
      message TEXT NOT NULL,
      additional_kwargs TEXT,  -- LangChain에서 필요할 수 있음 (역할, 모델 파라미터 ...)
      created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
  )
""")


conn.commit()
conn.close()


In [ ]:
import os
import pprint

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import AIMessage, HumanMessage
from langchain_community.chat_message_histories import SQLChatMessageHistory
from langchain.memory import ConversationSummaryMemory




# ================================
# 1. 요약에 사용될 LLM 모델 설정
# ================================

summary_model = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.0,
    openai_api_key=os.environ["OPENAI_API_KEY"]
)


# ================================
# 2. Summary Prompt & Memory
# ================================
summary_prompt = ChatPromptTemplate.from_template(
    """다음 대화를 한국어로 간결하고 이해하기 쉽게 요약하세요.
    요약 시 아래를 반드시 지켜주세요:
    1. 중요한 정보와 맥락을 포함할 것.
    2. 불필요한 반복이나 사소한 내용은 제거할 것.
    3. 가능한 한 자연스러운 한국어 문장으로 작성할 것.

    대화 내용:
    {summary}

    새로운 메시지:
    {new_lines}

    위 내용을 바탕으로 최신 대화까지 포함하여 한국어로 요약하세요."""
)

summary_memory = ConversationSummaryMemory(
    llm=summary_model,
    memory_key="summary",   # 메모리에 저장될 key 이름, 프롬프트에서 {summary}로  접근 가능
    return_messages=False,  # LM 출력 반환 형태 : False면 문자열 요약 반환, True면 AIMessage 객체로 반환
    prompt=summary_prompt   # 요약 생성용 커스텀 프롬프트
)

# ================================
# 4. 모델 설정
# ================================
model = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.3,
    openai_api_key=os.environ["OPENAI_API_KEY"]
)


# ================================
# 5. Chat Prompt Template
# ================================
prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 친절하고 도움이 되는 AI 비서입니다. "
               "아래의 summary(요약된 오래된 기록)와 "
               "recent chat_history(최신 대화 몇 개)를 참고하여 한국어로 답변하세요."),
    ("system", "요약된 이전 대화 내용:\n{summary}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}")
])


# ================================
# 6. LCEL 체인 생성
# ================================

chain = RunnablePassthrough() | prompt | model | StrOutputParser()


# ================================
# 5. respond() 함수
# ================================
def respond(session_id: str, human_message: str) -> AIMessage:

    print("\n--- 샤용자 입력 ---")
    pprint.pprint(human_message)
    print("------------------\n")


    # --- DB 기반 전체 대화 목록 로드 ---
    chat_history_store = SQLChatMessageHistory(
        connection_string=f"sqlite:///{DB_FILE}",
        session_id=session_id,
        table_name="message_store"
    )

    full_messages = chat_history_store.get_messages()

    print("\n--- 전체 DB 대화 기록 ---")
    pprint.pprint(full_messages)
    print("------------------\n")

    # --- Summary Memory 초기화 ---
    summary_memory.chat_memory.clear()  # 기존 메모리 초기화

    # --- chat_memory에 DB 메시지 추가 ---
    for m in full_messages:
        if isinstance(m, HumanMessage):
            summary_memory.chat_memory.add_user_message(m.content)
        elif isinstance(m, AIMessage):
            summary_memory.chat_memory.add_ai_message(m.content)


    # --- 첫 요약 강제 생성 ---
    for m in full_messages:
        if isinstance(m, HumanMessage):
            summary_memory.save_context(
                {"input": m.content},
                {"output": ""}  # 과거 AI 메시지는 없으므로 빈 문자열 사용
            )


    #  프롬프트에 넣을 요약 메시지 가져오기
    summary_vars = summary_memory.load_memory_variables({})

    print("\n--- 요약 데이터 ---")
    print(summary_vars.get("summary", ""))
    print("------------------\n")

    # --- 최신 3개 메시지만 LLM에 전달 ---
    recent_messages = full_messages[-3:]


    # --- LLM 호출 ---
    ai_message = chain.invoke({
        "summary": summary_vars.get("summary", ""),
        "chat_history": recent_messages,
        "input": human_message
    })

    # --- DB 저장 ---
    chat_history_store.add_user_message(human_message)
    chat_history_store.add_ai_message(ai_message)

    # --- Summary Memory 업데이트 ---
    summary_memory.save_context(
        {"input": human_message},
        {"output": ai_message}
    )

    return ai_message


# ================================
# 6. 실행 테스트
# ================================
if __name__ == "__main__":
    for msg in ["AI에 대해 설명해줘.", "더 자세히 설명해줘", "요약해줘"]:
        response = respond(session_id="session_1234", human_message=msg)
        print("\n--- AI 응답 ---")
        pprint.pprint(response)



--- 샤용자 입력 ---
'AI에 대해 설명해줘.'
------------------


--- 전체 DB 대화 기록 ---
[HumanMessage(content='AI가 무엇인지 기본 개념부터 설명해줘.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='AI는 인간의 학습·추론·판단 능력을 컴퓨터가 수행하도록 만드는 기술을 말합니다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='AI랑 머신러닝은 같은 거야?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='같지는 않아요. AI가 더 큰 개념이고, 머신러닝은 데이터로 패턴을 학습하는 AI의 한 분야입니다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='AI는 어떤 산업에서 주로 쓰이고 있어?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='의료, 금융, 제조, 교육, 자율주행, 음성 비서, 추천 시스템 등 거의 모든 산업에서 활용됩니다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='AI가 사람보다 잘하는 분야가 있어?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='대규모 데이터 분석, 패턴 탐지, 특정 반복 작업 등은 인간보다 훨씬 빠르고 정확하게 처리할 수 있습니다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='반대로 AI가 아직 부족한 부분은 뭐야?.', additional_kwargs={}, respons

KeyboardInterrupt: 

##RunnableWithMessageHistory

* RunnableWithMessageHistory는 Message-based memory만 자동 주입/저장하며, SummaryMemory는 수동으로 관리해야 한다.


In [ ]:
# Python 코드로 테이블 생성 (단, 한 번만 실행)

import sqlite3

DB_FILE = "chat_history.db"

conn = sqlite3.connect(DB_FILE)

cursor = conn.cursor()

cursor.execute("""
  CREATE TABLE IF NOT EXISTS message_store (
      id INTEGER PRIMARY KEY AUTOINCREMENT,
      session_id TEXT NOT NULL,
      message_type TEXT,
      message TEXT NOT NULL,
      additional_kwargs TEXT,  -- LangChain에서 필요할 수 있음 (역할, 모델 파라미터 ...)
      created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
  )
""")


conn.commit()
conn.close()

In [ ]:
import os
import pprint

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableWithMessageHistory
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_message_histories import SQLChatMessageHistory
from langchain.memory import ConversationSummaryMemory


DB_FILE = "chat_history.db"


# ---------------------------
# 1) 모델
# ---------------------------
model = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.3,
    openai_api_key=os.environ["OPENAI_API_KEY"]
)

summary_llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.0,
    openai_api_key=os.environ["OPENAI_API_KEY"]
)


# ---------------------------
# 2) 요약 메모리
# ---------------------------
summary_prompt = ChatPromptTemplate.from_template(
"""다음 대화를 한국어로 간결하게 요약하세요.

기존 요약:
{summary}

새 메시지:
{new_lines}

최신 요약:"""
)


summary_memory = ConversationSummaryMemory(
    llm=summary_llm,
    memory_key="summary",
    prompt=summary_prompt,
    return_messages=False
)


# ---------------------------
# 3) Chat Prompt
# ---------------------------
chat_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "당신은 친절한 AI 비서입니다.\n"
     "요약된 대화와 최근 히스토리를 참고하여 답변하세요."),
    ("system", "요약된 대화:\n{summary}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}")
])


# ---------------------------
# 4) DB 기반 히스토리 로딩
# ---------------------------
def get_history(session_id: str):
    return SQLChatMessageHistory(
        connection_string=f"sqlite:///{DB_FILE}",
        session_id=session_id,
        table_name="message_store"
    )


# ---------------------------
# 5) RunnableWithMessageHistory: chat_history만 자동 관리
# ---------------------------

chain = chat_prompt | model | StrOutputParser()

chat_runnable = RunnableWithMessageHistory(
    runnable=chain,
    get_session_history=get_history,
    history_messages_key="chat_history",  # MessagesPlaceholder
    input_messages_key="input",           # prompt의 {input}
    max_history=3
)


# ------------------------------------------------------
# 6) respond() — 요약 memory는 별도로 업데이트
# ------------------------------------------------------
def respond(session_id: str, user_input: str):

    # 1) 현재 summary 가져오기
    current_summary = summary_memory.load_memory_variables({}).get("summary", "")

    # 2) 모델 호출 (스트리밍)
    response = chat_runnable.invoke(
        {
            "summary": current_summary,
            "input": user_input
        },
        config={"configurable": {"session_id": session_id}}
    )

    # 3) DB 자동 저장은 RunnableWithMessageHistory가 처리
    # 4) 요약 갱신 — save_context 호출로 LLM 실행
    summary_memory.save_context(
        {"input": user_input},
        {"output": response}
    )

    return response


# ---------------------------
# 7) 실행 테스트
# ---------------------------
if __name__ == "__main__":
    messages = ["AI에 대해 설명해줘", "더 자세히 설명해줘", "요약해줘"]

    for msg in messages:
        print(f"\n=== 입력: {msg} ===")
        answer = respond("session_1234", msg)
        print("--- AI 응답 ---")
        print(answer)

        print("\n--- 최신 요약 ---")
        print(summary_memory.load_memory_variables({})["summary"])
